# Introduction to tf-shell

To get started, `pip install tf-shell`. tf-shell has a few modules, the one used
in this notebook is `tf_shell`.

In [1]:
import tf_shell

2024-06-10 21:54:19.630781: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 21:54:19.631217: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 21:54:19.633550: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 21:54:19.663933: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 21:54:20.301503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

First, set up some low level parameters for the BGV encryption scheme and
generate a secret key. If you don't want to set these parameters yourself, see
the `intro_with_auto_params.ipynb` notebook.

- `log_n` coresponds to the ring degree.
- `main_moduli` are a product of primes, less than 60 bits, that define the main
modulus. tf-shell uses RNS representation over each of these moduli.
- `scaling_factor` is used to encode fractional numbers, e.g. the number 1.5
with a scaling factor of 2 will be represented as 3.
- `mul_depth_supported` is for debugging purposes. The other parameters are
highly sensitive to the multiplication depth of the computation and tf-shell
will check that the multiplication depth is not exceeded during the computation.

In [2]:
context = tf_shell.create_context64(
    log_n=10,
    main_moduli=[8556589057, 8388812801],
    plaintext_modulus=40961,
    scaling_factor=3,
    mul_depth_supported=3,
    seed="test_seed",
)

secret_key = tf_shell.create_key64(context)

Next, encrypt something. tf-shell treats the outer dimension of all tensors as
the packing dimension of the BGV scheme. This is for efficiency purposes and, in
short, means the first dimension of all tensors is 2^log_n as defined above.

The cost of operating on a ciphertext (e.g. addition), regardless of how "full"
the first dimension is, is the same.

In [3]:
import tensorflow as tf
tf_data = tf.random.uniform([context.num_slots, 2], dtype=tf.float32, maxval=10)
print(f"The first 3 elements of the data are {tf_data[:3, 0]}")

enc = tf_shell.to_encrypted(tf_data, secret_key, context)

The first 3 elements of the data are [0.05918741 3.8001454  5.9336624 ]


Next, perform some homomorphic operations on the ciphertexts. tf-shell can
perform addition, subtraction, and multiplication on ciphertexts, shell
plaintexts, and tensorflow tensors.

In [4]:
# Ciphertext . scalar
ct_scalar_add = enc + 3
ct_scalar_mul = enc * 3

# Ciphertext . tensorflow tensor
three = tf.constant(3, dtype=tf.float32)
ct_tf_add = enc + three
ct_tf_mul = enc * three

# Ciphertext . plaintext
three_repeated = tf.ones([context.num_slots, 1], dtype=tf.float32) * 3
three_repeated = tf_shell.to_shell_plaintext(three_repeated, context)
ct_pt_add = enc + three_repeated
ct_pt_mul = enc * three_repeated

# ciphertext . ciphertext
ct_ct_add = enc + enc
ct_ct_mul = enc * enc

Finally, let's decrypt the results using the secret key.

In [5]:
dec = tf_shell.to_tensorflow(enc, secret_key)
add = tf_shell.to_tensorflow(ct_ct_add, secret_key)
mul = tf_shell.to_tensorflow(ct_ct_mul, secret_key)

print(f"enc:       {dec[:3, 0]}")
print(f"enc + enc: {add[:3, 0]}")
print(f"enc * enc: {mul[:3, 0]}")

enc:       [0.        3.6666667 6.       ]
enc + enc: [ 0.         7.3333335 12.       ]
enc * enc: [ 0.       13.444445 36.      ]
